In [15]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestClassifier
%matplotlib inline

#參考文章：https://www.kaggle.com/nishimoto/very-simple-linear-models-pls

In [16]:
train = pd.read_csv("train_new.csv")
test = pd.read_csv("test.csv")


In [17]:
train = pd.read_csv('train_new.csv')
test = pd.read_csv('test.csv')
testID = test['id']

#data = pd.concat([train.drop('SalePrice', axis=1), test], keys=['train', 'test'])
#data.drop(['Id'], axis=1, inplace=True)
#df_all = pd.concat([train.drop('SalePrice', axis=1), test], keys=['train', 'test'])
#df_all.drop(['Id'], axis=1, inplace=True)

In [18]:
print(f"train shape : {train.shape}")
print(f" test shape : {test.shape}")

train shape : (7160, 60)
 test shape : (1791, 183)


In [19]:
#瞭解資料型別
#train.info()

In [20]:
#train.corr('kendall')

In [21]:
# 描述data物件型態的統計資訊（個數、種類、出現頻率最高的種類及其次數）
print(f"categorical features: {train.describe(include='O').shape}")
# 查詢非數值資訊
train.describe(include="O")

categorical features: (4, 6)


,Id,appearedTimeOfDay,city,continent,weather,weatherIcon
count,7160,7160,7160,7160,7160,7160
unique,7160,4,73,8,24,8
top,NjM1NzQwOTgzNzI2NDYwNzg4NQ==,night,New_York,America,Clear,clear-day
freq,1,3781,1387,4029,4319,2263


## EDA (Drop Outlier)

In [22]:
# 描述data數值型態的統計資訊（個數、平均、標準差、最小值、
#                           第一四分位數、第二四分位數、第三四分位數
#                           、最大值）
     
# 瞭解資料的分佈情況
train.describe()

,terrainType,temperature,windSpeed,pressure,population_density,gymDistanceKm,pokestopDistanceKm,class
count,7160.00000,7160.000000,7160.000000,7160.000000,7160.000000,7160.000000,7160.000000,7160.000000
mean,9.76676,23.276257,7.536071,1014.900068,1532.785042,2.849919,0.468334,2.374581
std,4.90665,6.042675,4.622314,5.845140,2216.693146,50.348234,3.319317,1.806051
min,0.00000,4.800000,0.000000,996.330000,0.000000,0.004516,0.000319,0.000000
25%,7.00000,18.500000,3.930000,1010.660000,86.498360,0.177619,0.050233,1.000000
50%,13.00000,23.800000,6.650000,1015.290000,635.549800,0.309457,0.131560,2.000000
75%,13.00000,27.500000,9.990000,1019.320000,1955.863600,0.576796,0.305844,4.000000
max,16.00000,41.200000,56.130000,1034.170000,10000.000000,3859.424664,199.829897,5.000000


In [23]:
## EDA (Drop Outlier)

In [24]:
train.shape

(7160, 60)

In [25]:
# 一些右偏分佈的 feature，可透過取 log 將其轉為常態分佈
#sns.distplot(np.log(train.class))

In [26]:
#scatterplot
# 畫出散狀圖, 'GarageCars', 'TotalBsmtSF', 'FullBath', 'OverallQual',
#sns.set()
#cols = ['SalePrice', 'GrLivArea' , 'YearBuilt']
#sns.pairplot(train[cols], size = 5)
#plt.show();

In [27]:
#取得數值型態的欄位
#取得數值型態欄位
numeric_data = train._get_numeric_data()
numeric_data.columns

Index(['terrainType', 'closeToWater', 'temperature', 'windSpeed', 'pressure',
       'population_density', 'urban', 'suburban', 'midurban', 'rural',
       'gymDistanceKm', 'gymIn100m', 'gymIn250m', 'gymIn500m', 'gymIn1000m',
       'gymIn2500m', 'gymIn5000m', 'pokestopDistanceKm', 'pokestopIn100m',
       'pokestopIn250m', 'pokestopIn500m', 'pokestopIn1000m',
       'pokestopIn2500m', 'pokestopIn5000m', 'cooc_1', 'cooc_2', 'cooc_3',
       'cooc_4', 'cooc_5', 'cooc_6', 'cooc_7', 'cooc_8', 'cooc_9', 'cooc_10',
       'cooc_11', 'cooc_12', 'cooc_13', 'cooc_14', 'cooc_15', 'cooc_16',
       'cooc_17', 'cooc_18', 'cooc_19', 'cooc_20', 'cooc_21', 'cooc_22',
       'cooc_23', 'cooc_24', 'cooc_25', 'cooc_26', 'cooc_27', 'cooc_28',
       'cooc_29', 'class'],
      dtype='object')

## sketch pics


## Drop Null Feature(s)

In [28]:
#df_all = pd.concat( [ train.drop( [ 'Id', 'SalePrice' ], axis = 1 ),
#                      test.drop( [ 'Id' ], axis = 1 ) ],
#                      axis = 0, ignore_index = False )
train.drop_duplicates(keep='first', inplace=False) 
test.drop_duplicates(keep='first', inplace=False) 
#train_data=train.drop('class', axis=1)
df_all = pd.concat([train, test], keys=['train', 'test'], sort=True)


In [29]:
df_all.drop(['id'], axis=1, inplace=True)

In [30]:
# check null status
print( df_all.shape )
df_na = df_all.select_dtypes( exclude = [ 'object' ] ).isnull().sum()
print( df_na[ df_na > 100 ].sort_values(ascending=False) )

(8951, 184)
appearedMinute    7160
appearedHour      7160
class             1791
dtype: int64


## Numerical to Categorical

In [31]:
# handle skewness
#train.SalePrice = np.log( train['class'] )
# exclude object型態的欄位
#num_feats = df_all.select_dtypes( exclude = [ 'object' ] ).columns
# na drop should be done series by series
# apply function>>幫助我們算出偏差系數
#skewed_feats = df_all[ num_feats ].apply( func = lambda s: skew(s.dropna()), axis = 0 )
#print(skewed_feats)
# 選出dataframe大於0.75的人
#skewed_feats = skewed_feats[ skewed_feats > 0.75 ].index
# print(skewed_feats)
# 把大於0.75的要作正規化，變成常態分配
# np.log1p是log(1+x), 若直接取log會有log0的問題
#df_all[ skewed_feats ] = np.log1p( df_all[ skewed_feats ] )



In [32]:
df_all.describe(include = "O")

,Id,appearedTimeOfDay,city,continent,cooc_100,cooc_101,cooc_102,cooc_103,cooc_104,cooc_105,...,cooc_92,cooc_93,cooc_94,cooc_95,cooc_96,cooc_97,cooc_98,cooc_99,weather,weatherIcon
count,7160,8951,8951,8951,1791,1791,1791,1791,1791,1791,...,1791,1791,1791,1791,1791,1791,1791,1791,8951,8951
unique,7160,4,75,9,2,2,2,1,2,2,...,2,1,1,2,2,1,2,2,25,8
top,NjM1NzQwOTgzNzI2NDYwNzg4NQ==,night,New_York,America,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,Clear,clear-day
freq,1,4739,1724,5041,1749,1787,1783,1791,1777,1790,...,1777,1791,1791,1790,1545,1791,1761,1788,5431,2823


## One-Hot Encoding

In [33]:
df_all_dum = pd.get_dummies(df_all); df_all_dum.head()

appearedHour  appearedMinute  class  closeToWater  cooc_1  cooc_10  \
train 0           NaN             NaN    2.0         False   False     True   
      1           NaN             NaN    5.0          True   False    False   
      2           NaN             NaN    3.0          True   False    False   
      3           NaN             NaN    4.0          True   False    False   
      4           NaN             NaN    3.0          True   False    False   

         cooc_11  cooc_12  cooc_13  cooc_14        ...         \
train 0    False    False    False    False        ...          
      1    False    False    False    False        ...          
      2    False    False    False    False        ...          
      3    False    False    False    False        ...          
      4    False    False    False    False        ...          

         weather_WindyandFoggy  weather_WindyandPartlyCloudy  \
train 0                      0                             0   
      1                      0                             0   
      2                      0                             0   
      3                      0                             0   
      4                      0                             0   

         weatherIcon_clear-day  weatherIcon_clear-night  weatherIcon_cloudy  \
train 0                      1                        0                   0   
      1                      0                        1                   0   
      2                      1                        0                   0   
      3                      0                        1                   0   
      4                      0                        0                   0   

         weatherIcon_fog  weatherIcon_partly-cloudy-day  \
train 0                0                              0   
      1                0                              0   
      2                0                              0   
      3                0                              0   
      4                0                              1   

         weatherIcon_partly-cloudy-night  weatherIcon_rain  weatherIcon_wind  
train 0                                0                 0                 0  
      1                                0                 0                 0  
      2                                0                 0                 0  
      3                                0                 0                 0  
      4                                0                 0                 0  

[5 rows x 7541 columns]

In [34]:
# 將NA的值填平均值
df_all_dum = df_all_dum.fillna( df_all_dum.mean() ); df_all_dum.head()

appearedHour  appearedMinute  class  closeToWater  cooc_1  cooc_10  \
train 0     10.414294       30.599665    2.0         False   False     True   
      1     10.414294       30.599665    5.0          True   False    False   
      2     10.414294       30.599665    3.0          True   False    False   
      3     10.414294       30.599665    4.0          True   False    False   
      4     10.414294       30.599665    3.0          True   False    False   

         cooc_11  cooc_12  cooc_13  cooc_14        ...         \
train 0    False    False    False    False        ...          
      1    False    False    False    False        ...          
      2    False    False    False    False        ...          
      3    False    False    False    False        ...          
      4    False    False    False    False        ...          

         weather_WindyandFoggy  weather_WindyandPartlyCloudy  \
train 0                      0                             0   
      1                      0                             0   
      2                      0                             0   
      3                      0                             0   
      4                      0                             0   

         weatherIcon_clear-day  weatherIcon_clear-night  weatherIcon_cloudy  \
train 0                      1                        0                   0   
      1                      0                        1                   0   
      2                      1                        0                   0   
      3                      0                        1                   0   
      4                      0                        0                   0   

         weatherIcon_fog  weatherIcon_partly-cloudy-day  \
train 0                0                              0   
      1                0                              0   
      2                0                              0   
      3                0                              0   
      4                0                              1   

         weatherIcon_partly-cloudy-night  weatherIcon_rain  weatherIcon_wind  
train 0                                0                 0                 0  
      1                                0                 0                 0  
      2                                0                 0                 0  
      3                                0                 0                 0  
      4                                0                 0                 0  

[5 rows x 7541 columns]

In [35]:
## Standardization

## Standardization

In [36]:
# normalization helps a bit
from sklearn import preprocessing

num_feats = df_all_dum.select_dtypes( exclude = [ 'object' ] ).columns
df_all_dum[ num_feats ] = preprocessing.scale( df_all_dum[ num_feats ] )
df_all_dum[ num_feats ].head()

appearedHour  appearedMinute     class  closeToWater    cooc_1  \
train 0 -5.646535e-16    1.383354e-15 -0.231913     -0.901079 -0.083516   
      1 -5.646535e-16    1.383354e-15  1.625467      1.109781 -0.083516   
      2 -5.646535e-16    1.383354e-15  0.387214      1.109781 -0.083516   
      3 -5.646535e-16    1.383354e-15  1.006340      1.109781 -0.083516   
      4 -5.646535e-16    1.383354e-15  0.387214      1.109781 -0.083516   

          cooc_10   cooc_11  cooc_12   cooc_13   cooc_14        ...         \
train 0  6.597904 -0.049638 -0.01495 -0.221883 -0.050756        ...          
      1 -0.151563 -0.049638 -0.01495 -0.221883 -0.050756        ...          
      2 -0.151563 -0.049638 -0.01495 -0.221883 -0.050756        ...          
      3 -0.151563 -0.049638 -0.01495 -0.221883 -0.050756        ...          
      4 -0.151563 -0.049638 -0.01495 -0.221883 -0.050756        ...          

         weather_WindyandFoggy  weather_WindyandPartlyCloudy  \
train 0               -0.01831                      -0.01057   
      1               -0.01831                      -0.01057   
      2               -0.01831                      -0.01057   
      3               -0.01831                      -0.01057   
      4               -0.01831                      -0.01057   

         weatherIcon_clear-day  weatherIcon_clear-night  weatherIcon_cloudy  \
train 0               1.473343                -0.625635           -0.221051   
      1              -0.678728                 1.598376           -0.221051   
      2               1.473343                -0.625635           -0.221051   
      3              -0.678728                 1.598376           -0.221051   
      4              -0.678728                -0.625635           -0.221051   

         weatherIcon_fog  weatherIcon_partly-cloudy-day  \
train 0        -0.073426                      -0.495947   
      1        -0.073426                      -0.495947   
      2        -0.073426                      -0.495947   
      3        -0.073426                      -0.495947   
      4        -0.073426                       2.016345   

         weatherIcon_partly-cloudy-night  weatherIcon_rain  weatherIcon_wind  
train 0                        -0.341325          -0.18069         -0.135338  
      1                        -0.341325          -0.18069         -0.135338  
      2                        -0.341325          -0.18069         -0.135338  
      3                        -0.341325          -0.18069         -0.135338  
      4                        -0.341325          -0.18069         -0.135338  

[5 rows x 7541 columns]

In [37]:
np.any(np.isnan(df_all_dum))
df_all_dum.columns[df_all_dum.isna().any()].tolist()

[]

In [38]:
# 查詢是否有重複的欄位
df_all_dum.columns[df_all_dum.columns.duplicated()]

Index([], dtype='object')

## Training Start

## XGBoost

In [39]:
import tensorflow as tf

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [40]:
# inputs
from sklearn.model_selection import train_test_split
from matplotlib import rcParams
X = df_all_dum[:train.shape[0]]
y = train['class']
#把train data與test分開
#X, X_val, y, y_val = train_test_split(X, y, test_size = 0.1, random_state = 40)

X_test = df_all_dum[ train.shape[0]: ]


x_train, x_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.05, random_state = 40)

In [41]:
tf.reset_default_graph()

inputs = tf.placeholder(dtype=tf.float32, shape=(None, 784), name='x')
labels = tf.placeholder(dtype=tf.float32, shape=(None, 10), name='y')

# with tf.variable_scope('hidden_layer'):
w1 = tf.Variable(tf.truncated_normal([784, 128]), name='w1')
b1 = tf.Variable(tf.zeros(128), name='b1')
z1 = tf.matmul(inputs, w1) + b1
a1 = tf.nn.relu(z1)
#     a1 = tf.layers.dense(inputs=inputs, units=128, activation=tf.nn.relu, name='w1')

# with tf.variable_scope('output_layer'):
w2 = tf.Variable(tf.truncated_normal([128, 10]), name='w2')
b2 = tf.Variable(tf.zeros(10), name='b2')
z2 = tf.matmul(a1, w2) + b2
#     z2 = tf.layers.dense(inputs=a1, units=10, name='w2')
    
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=labels, logits=z2), name='loss')
optimizer = tf.train.AdamOptimizer()
train_step = optimizer.minimize(loss)

predict_booling = tf.equal(tf.argmax(labels, axis=1), tf.argmax(z2, axis=1))
accuracy = tf.reduce_mean(tf.cast(predict_booling, tf.float32))

In [42]:
writer = tf.summary.FileWriter('Tensorboard/', graph=tf.get_default_graph())

In [43]:
loss

<tf.Tensor 'loss:0' shape=() dtype=float32>

In [44]:
tf.global_variables()

[<tf.Variable 'w1:0' shape=(784, 128) dtype=float32_ref>,
 <tf.Variable 'b1:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'w2:0' shape=(128, 10) dtype=float32_ref>,
 <tf.Variable 'b2:0' shape=(10,) dtype=float32_ref>,
 <tf.Variable 'beta1_power:0' shape=() dtype=float32_ref>,
 <tf.Variable 'beta2_power:0' shape=() dtype=float32_ref>,
 <tf.Variable 'w1/Adam:0' shape=(784, 128) dtype=float32_ref>,
 <tf.Variable 'w1/Adam_1:0' shape=(784, 128) dtype=float32_ref>,
 <tf.Variable 'b1/Adam:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'b1/Adam_1:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'w2/Adam:0' shape=(128, 10) dtype=float32_ref>,
 <tf.Variable 'w2/Adam_1:0' shape=(128, 10) dtype=float32_ref>,
 <tf.Variable 'b2/Adam:0' shape=(10,) dtype=float32_ref>,
 <tf.Variable 'b2/Adam_1:0' shape=(10,) dtype=float32_ref>]

In [52]:
from sklearn.utils import shuffle
from tqdm import tqdm_notebook

In [53]:
train.shape

(7160, 60)

In [66]:
from tensorflow.keras.layers import Dense
model = tf.keras.Sequential()
model.add(Dense(25, activation='relu'))
model.add(Dense(10, activation='softmax')) 

print(model.summary())

ValueError: This model has never been called, thus its weights have not yet been created, so no summary can be displayed. Build the model first (e.g. by calling it on some data).

In [63]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [64]:
batch_size = 32
epochs = 40
lr = 0.01
train_ratio = 0.9

In [65]:
model_history = model.fit(x=x_train, y=y_train,
                          batch_size=batch_size,
                          epochs=epochs,
                          validation_split=1 - train_ratio)

ValueError: Error when checking input: expected dense_6_input to have shape (754,) but got array with shape (7541,)

In [45]:
tf.reset_default_graph()

model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation=tf.nn.relu, input_shape=(784,)),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [46]:
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)

Epoch 1/10
60000/60000 [==============================] - 4s 62us/step - loss: 0.5004 - acc: 0.8238
Epoch 2/10
60000/60000 [==============================] - 4s 61us/step - loss: 0.3757 - acc: 0.8643
Epoch 3/10
60000/60000 [==============================] - 4s 60us/step - loss: 0.3392 - acc: 0.8767
Epoch 4/10
60000/60000 [==============================] - 4s 59us/step - loss: 0.3128 - acc: 0.8845
Epoch 5/10
60000/60000 [==============================] - 4s 60us/step - loss: 0.2937 - acc: 0.8910
Epoch 6/10
60000/60000 [==============================] - 4s 61us/step - loss: 0.2807 - acc: 0.8972
Epoch 7/10
60000/60000 [==============================] - 4s 60us/step - loss: 0.2688 - acc: 0.9005
Epoch 8/10
60000/60000 [==============================] - 4s 59us/step - loss: 0.2564 - acc: 0.9048
Epoch 9/10
60000/60000 [==============================] - 4s 59us/step - loss: 0.2486 - acc: 0.9070
Epoch 10/10
60000/60000 [==============================] - 4s 60us/step - loss: 0.2371 - acc: 0.9120

In [47]:
model.evaluate(X_test, y_test)

10000/10000 [==============================] - 0s 24us/step


[0.3579446482658386, 0.8776]

In [48]:
tf.global_variables()

[<tf.Variable 'dense/kernel:0' shape=(784, 128) dtype=float32>,
 <tf.Variable 'dense/bias:0' shape=(128,) dtype=float32>,
 <tf.Variable 'dense_1/kernel:0' shape=(128, 10) dtype=float32>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32>,
 <tf.Variable 'Adam/iterations:0' shape=() dtype=int64>,
 <tf.Variable 'Adam/lr:0' shape=() dtype=float32>,
 <tf.Variable 'Adam/beta_1:0' shape=() dtype=float32>,
 <tf.Variable 'Adam/beta_2:0' shape=() dtype=float32>,
 <tf.Variable 'Adam/decay:0' shape=() dtype=float32>,
 <tf.Variable 'training/Adam/Variable:0' shape=(784, 128) dtype=float32>,
 <tf.Variable 'training/Adam/Variable_1:0' shape=(128,) dtype=float32>,
 <tf.Variable 'training/Adam/Variable_2:0' shape=(128, 10) dtype=float32>,
 <tf.Variable 'training/Adam/Variable_3:0' shape=(10,) dtype=float32>,
 <tf.Variable 'training/Adam/Variable_4:0' shape=(784, 128) dtype=float32>,
 <tf.Variable 'training/Adam/Variable_5:0' shape=(128,) dtype=float32>,
 <tf.Variable 'training/Adam/Variable_6:0'